In [11]:
import pandas as pd
import os
import re
import time
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from wordsegment import load, segment
import emoji
from preprocessing import process_single_tweet, process_train_data, process_test_data
from utils import load_train_data, load_test_data_a

In [2]:
train_data = load_train_data()
train_data.head()

number of training data: 13240


,tweet,subtask_a
0,@USER She should ask a few native Americans wh...,OFF
1,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF
2,Amazon is investigating Chinese employees who ...,NOT
3,"@USER Someone should'veTaken"" this piece of sh...",OFF
4,@USER @USER Obama wanted liberals &amp; illega...,NOT


## Tokenizer

Result from NLTK twitter tokenizer:

In [3]:
example = train_data["tweet"][1]
print(example)
twt_tokenizer = TweetTokenizer()
tokenized = twt_tokenizer.tokenize(example)
print(tokenized)

@USER @USER Go home you’re drunk!!! @USER #MAGA #Trump2020 👊🇺🇸👊 URL
['@USER', '@USER', 'Go', 'home', 'you', '’', 're', 'drunk', '!', '!', '!', '@USER', '#MAGA', '#Trump2020', '👊', '🇺', '🇸', '👊', 'URL']


Result from our tokenizer:

In [4]:
print(example)
print(process_single_tweet(example))

@USER @USER Go home you’re drunk!!! @USER #MAGA #Trump2020 👊🇺🇸👊 URL
go home you are drunk ! ! ! maga trump 2020 oncoming fist united states oncoming fist
